In [34]:
import pandas as pd
import numpy as np
import yfinance as yf
import arch
from arch.unitroot import ADF
from arch import arch_model
import matplotlib.pyplot as plt
from copy import copy
import fattails
import datetime

In [2]:
seel = yf.download(tickers="SEEL", period="7d", interval="1m")
zyne = yf.download(tickers="ZYNE", period="7d", interval="1m")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
seel_adf = ADF(seel["Close"])
print(seel_adf.summary().as_text())

   Augmented Dickey-Fuller Results   
Test Statistic                 -1.139
P-value                         0.699
Lags                               25
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [4]:
zyne_adf = ADF(zyne["Close"])
print(zyne_adf.summary().as_text())

   Augmented Dickey-Fuller Results   
Test Statistic                 -1.950
P-value                         0.309
Lags                                1
-------------------------------------

Trend: Constant
Critical Values: -3.44 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [5]:
zyne_returns = 100 * (zyne["Close"].pct_change().dropna())
zyne_arch = arch_model(zyne_returns, dist="skewt")
zyne_res = zyne_arch.fit()
print(zyne_res.summary())

Iteration:      1,   Func. Count:      8,   Neg. LLF: 220091.95544456024
Iteration:      2,   Func. Count:     17,   Neg. LLF: 70926.094882721
Iteration:      3,   Func. Count:     27,   Neg. LLF: 22798.048991450658
Iteration:      4,   Func. Count:     36,   Neg. LLF: 1258.8354779109898
Iteration:      5,   Func. Count:     44,   Neg. LLF: 431.80195936292176
Iteration:      6,   Func. Count:     52,   Neg. LLF: 429.57496263101797
Iteration:      7,   Func. Count:     61,   Neg. LLF: 412.6387353001295
Iteration:      8,   Func. Count:     69,   Neg. LLF: 411.9327190602755
Iteration:      9,   Func. Count:     76,   Neg. LLF: 411.91458521836716
Iteration:     10,   Func. Count:     83,   Neg. LLF: 411.90487393306614
Iteration:     11,   Func. Count:     90,   Neg. LLF: 411.9045583167825
Iteration:     12,   Func. Count:     97,   Neg. LLF: 411.9045512788059
Iteration:     13,   Func. Count:    103,   Neg. LLF: 411.90455127888055
Optimization terminated successfully    (Exit mode 0)
    

In [6]:
zyne[-117:]

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2022-04-14 09:30:00-04:00,1.7800,1.7800,1.7700,1.7700,1.7700,3267
2022-04-14 09:36:00-04:00,1.7600,1.7700,1.7600,1.7600,1.7600,7380
2022-04-14 09:37:00-04:00,1.7600,1.7600,1.7500,1.7500,1.7500,2123
2022-04-14 09:38:00-04:00,1.7450,1.7500,1.7436,1.7436,1.7436,900
2022-04-14 09:39:00-04:00,1.7500,1.7500,1.7500,1.7500,1.7500,1300
...,...,...,...,...,...,...
2022-04-14 15:56:00-04:00,1.7210,1.7210,1.7200,1.7200,1.7200,910
2022-04-14 15:57:00-04:00,1.7285,1.7285,1.7285,1.7285,1.7285,158
2022-04-14 15:58:00-04:00,1.7200,1.7250,1.7200,1.7250,1.7250,1541


In [7]:
fig = zyne_res.plot(scale=252*120)

In [8]:
zyne_res.conditional_volatility

Datetime
2022-04-06 09:31:00-04:00    0.561956
2022-04-06 09:34:00-04:00    0.682392
2022-04-06 09:36:00-04:00    0.733078
2022-04-06 09:37:00-04:00    0.625374
2022-04-06 09:38:00-04:00    0.544945
                               ...   
2022-04-14 15:56:00-04:00    0.382636
2022-04-14 15:57:00-04:00    0.352803
2022-04-14 15:58:00-04:00    0.367084
2022-04-14 15:59:00-04:00    0.346847
2022-04-14 16:00:00-04:00    0.339403
Name: cond_vol, Length: 1260, dtype: float64

In [9]:
zyne_forecasts = zyne_res.forecast(reindex=False)
print(zyne_forecasts.mean.iloc[-3:])
print(zyne_forecasts.residual_variance.iloc[-3:])
print(zyne_forecasts.variance.iloc[-3:])

                                h.1
Datetime                           
2022-04-14 16:00:00-04:00 -0.009247
                                h.1
Datetime                           
2022-04-14 16:00:00-04:00  0.136057
                                h.1
Datetime                           
2022-04-14 16:00:00-04:00  0.136057


In [10]:
zyne_forecasts.variance["h.1"][0]

0.1360568071383528

In [11]:
np.sqrt(0.136057)

0.3688590516715023

In [12]:
def next_min_bound(ticker, st_devs):
    today = datetime.datetime.now()
    four = (today - datetime.timedelta(days = 28)).date().strftime("%Y-%m-%d")
    three = (today - datetime.timedelta(days = 21)).date().strftime("%Y-%m-%d")
    two = (today - datetime.timedelta(days = 14)).date().strftime("%Y-%m-%d")
    one = (today - datetime.timedelta(days = 7)).date().strftime("%Y-%m-%d")
    t43 = yf.download(tickers=ticker, start=four, end=three, interval="1m")
    t32 = yf.download(tickers=ticker, start=three, end=two, interval="1m")
    t21 = yf.download(tickers=ticker, start=two, end=one, interval="1m")
    t10 = yf.download(tickers=ticker, period="7d", interval="1m")
    ticker = pd.concat([t43, t32, t21, t10])
    adf = ADF(ticker["Close"])
    print(adf.summary().as_text())
    returns = 100 * (ticker["Close"].pct_change().dropna())
    arch = arch_model(returns, dist="skewt")
    result = arch.fit()
    print(result.summary())
    print(result.conditional_volatility)
    var = result.forecast(reindex=False).variance["h.1"][0]
    return np.sqrt(var)*st_devs

In [13]:
def full_month(ticker):
    today = datetime.datetime.now()
    four = (today - datetime.timedelta(days = 28)).date().strftime("%Y-%m-%d")
    three = (today - datetime.timedelta(days = 21)).date().strftime("%Y-%m-%d")
    two = (today - datetime.timedelta(days = 14)).date().strftime("%Y-%m-%d")
    one = (today - datetime.timedelta(days = 7)).date().strftime("%Y-%m-%d")
    t43 = yf.download(tickers=ticker, start=four, end=three, interval="1m")
    t32 = yf.download(tickers=ticker, start=three, end=two, interval="1m")
    t21 = yf.download(tickers=ticker, start=two, end=one, interval="1m")
    t10 = yf.download(tickers=ticker, period="7d", interval="1m")
    ticker = pd.concat([t43, t32, t21, t10])
    return ticker

In [14]:
def adf_full_month(ticker):
    tick = full_month(ticker)
    adf = ADF(tick["Close"])
    print(adf.summary().as_text())

In [15]:
def conditional_vol(ticker):
    ticker = full_month(ticker)
    returns = 10000 * (ticker["Close"].pct_change().dropna())
    arch = arch_model(returns, dist="skewt")
    result = arch.fit()
    return ticker, returns, result.conditional_volatility

In [16]:
sedg, sedg_returns, sedg_vol = conditional_vol("SEDG")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 179334.7801958855
Iteration:      2,   Func. Count:     16,   Neg. LLF: 51893.70101534875
Iteration:      3,   Func. Count:     24,   Neg. LLF: 43958.50736040053
Iteration:      4,   Func. Count:     32,   Neg. LLF: 62647.668142682174
Iteration:      5,   Func. Count:     40,   Neg. LLF: 42536.803304422836
Iteration:      6,   Func. Count:     48,   Neg. LLF: 38585.59934077563
Iteration:      7,   Func. Count:     56,   Neg. LLF: 37651.45062640076
Iteration:      8,   Func. Count:     64,   Neg. LLF: 36617.834929578676
Iteration:      9,   Func. Count:     72,   Neg. LLF: 33550.891994949496
Iteration:     10,   Func. Count:     79,   Neg. LLF: 33536.12611864341


In [17]:
sedg_vol.plot()

<AxesSubplot:title={'center':'Annualized Conditional Volatility'}, xlabel='Datetime'>

In [18]:
enph, enph_returns, enph_vol = conditional_vol("ENPH")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 225697.99830909495
Iteration:      2,   Func. Count:     16,   Neg. LLF: 81592.9493977958
Iteration:      3,   Func. Count:     25,   Neg. LLF: 324729.5434742085
Iteration:      4,   Func. Count:     34,   Neg. LLF: 1877862.2553000455
Iteration:      5,   Func. Count:     43,   Neg. LLF: 36287.30142811258
Iteration:      6,   Func. Count:     51,   Neg. LLF: 36158.216726873696
Iteration:      7,   Func. Count:     59,   Neg. LLF: 36100.07316297298
Iteration:      8,   Func. Count:     67,   Neg. LLF: 35942.13915662142
Iteration:      9,   Func. Count:     74,   Neg. LLF: 40068.31402850847
Iteration:     10,   Func. Count:     83,   Neg. LLF: 49119.48815072127
It

In [19]:
sedg_differential = sedg_returns + sedg_vol
sedg_times = sedg_differential[sedg_differential < 0].reset_index()

In [20]:
enph = enph.reset_index().reset_index()

In [21]:
for i in range(1, 30):
    trimmed = enph[:-i]
    entries = trimmed.loc[trimmed["Datetime"].isin(sedg_times["Datetime"])].reset_index(drop=True)
    sell = enph.loc[enph["index"].isin(entries["index"] + i)]
    profit = sum(sell["Close"]) - sum(entries["Close"])
    print(profit)

-29.537643432617188
-31.748138427734375
-56.65745544433594
-73.71771240234375
-33.88264465332031
-23.820068359375
6.328826904296875
-10.430313110351562
-13.881515502929688
14.733978271484375
10.561248779296875
17.05963134765625
22.526107788085938
27.486419677734375
32.384674072265625
23.651260375976562
41.92082214355469
57.7249755859375
59.832672119140625
60.99626159667969
36.04844665527344
52.60725402832031
74.9349365234375
51.03712463378906
52.73712158203125
81.12176513671875
49.65742492675781
50.42021179199219
51.76837158203125


In [22]:
def t1_to_t2_test(ticker1, ticker2, window):
    t1, t1_returns, t1_vol = conditional_vol(ticker1)
    t2, t2_returns, t2_vol = conditional_vol(ticker2)
    t1_diff = t1_returns + t1_vol
    t1_times = t1_diff[t1_diff < 0].reset_index()
    t2 = t2.reset_index().reset_index()
    for i in range(1, window + 1):
        trimmed = t2[:-i]
        entries = trimmed.loc[trimmed["Datetime"].isin(t1_times["Datetime"])].reset_index(drop=True)
        sell = t2.loc[t2["index"].isin(entries["index"] + i)]
        profit = sum(sell["Close"]) - sum(entries["Close"])
        print(f"{profit} profit when held for {i} minutes")

In [23]:
adf_full_month("ENPH")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
   Augmented Dickey-Fuller Results   
Test Statistic                 -2.387
P-value                         0.145
Lags                               14
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [24]:
t1_to_t2_test("SEDG", "ENPH", 30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 179334.7801958855
Iteration:      2,   Func. Count:     16,   Neg. LLF: 51893.70101534875
Iteration:      3,   Func. Count:     24,   Neg. LLF: 43958.50736040053
Iteration:      4,   Func. Count:     32,   Neg. LLF: 62647.668142682174
Iteration:      5,   Func. Count:     40,   Neg. LLF: 42536.803304422836
Iteration:      6,   Func. Count:     48,   Neg. LLF: 38585.59934077563
Iteration:      7,   Func. Count:     56,   Neg. LLF: 37651.45062640076
Iteration:      8,   Func. Count:     64,   Neg. LLF: 36617.834929578676
Iteration:      9,   Func. Count:     72,   Neg. LLF: 33550.891994949496
Iteration:     10,   Func. Count:     79,   Neg. LLF: 33536.12611864341


In [25]:
ctva = full_month("CTVA")
ctva_adf = ADF(ctva["Close"])
print(ctva_adf.summary().as_text())

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
   Augmented Dickey-Fuller Results   
Test Statistic                 -1.972
P-value                         0.299
Lags                               17
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [26]:
t1_to_t2_test("NTR", "CTVA", 30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 140775.83127859645
Iteration:      2,   Func. Count:     16,   Neg. LLF: 428637.1449682371
Iteration:      3,   Func. Count:     24,   Neg. LLF: 58951.17981562015
Iteration:      4,   Func. Count:     32,   Neg. LLF: 499752.5266479044
Iteration:      5,   Func. Count:     40,   Neg. LLF: 45362.46999346875
Iteration:      6,   Func. Count:     48,   Neg. LLF: 193426.33872120394
Iteration:      7,   Func. Count:     56,   Neg. LLF: 67778.74121869405
Iteration:      8,   Func. Count:     64,   Neg. LLF: 41509.61331907492
Iteration:      9,   Func. Count:     72,   Neg. LLF: 43724.14656916351
Iteration:     10,   Func. Count:     80,   Neg. LLF: 35298.42052200378
It

In [27]:
t1_to_t2_test("CTVA", "NTR", 30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 218020.9437950575
Iteration:      2,   Func. Count:     16,   Neg. LLF: 193861.51761118812
Iteration:      3,   Func. Count:     25,   Neg. LLF: 409299.1142148691
Iteration:      4,   Func. Count:     33,   Neg. LLF: 37207.54491083576
Iteration:      5,   Func. Count:     41,   Neg. LLF: 34559.53684752994
Iteration:      6,   Func. Count:     49,   Neg. LLF: 28644.28222833287
Iteration:      7,   Func. Count:     57,   Neg. LLF: 31952.994900119582
Iteration:      8,   Func. Count:     65,   Neg. LLF: 28367.47389862084
Iteration:      9,   Func. Count:     73,   Neg. LLF: 28302.24743389761
Iteration:     10,   Func. Count:     81,   Neg. LLF: 28302.082252227636
I

In [28]:
t1_to_t2_test("ADM", "BG", 30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 200210.96784668078
Iteration:      2,   Func. Count:     16,   Neg. LLF: 207316.8081145289
Iteration:      3,   Func. Count:     24,   Neg. LLF: 295681.2803056722
Iteration:      4,   Func. Count:     32,   Neg. LLF: 69138.50002995072
Iteration:      5,   Func. Count:     40,   Neg. LLF: 45031.60524825537
Iteration:      6,   Func. Count:     48,   Neg. LLF: 28757.3495248366
Iteration:      7,   Func. Count:     56,   Neg. LLF: 32850.77075887865
Iteration:      8,   Func. Count:     64,   Neg. LLF: 28571.368975333306
Iteration:      9,   Func. Count:     72,   Neg. LLF: 32964.73109608979
Iteration:     10,   Func. Count:     80,   Neg. LLF: 28515.858155932565
It

In [29]:
t1_to_t2_test("BG", "ADM", 30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 83447.10824048043
Iteration:      2,   Func. Count:     16,   Neg. LLF: 391527.9178820209
Iteration:      3,   Func. Count:     24,   Neg. LLF: 376000.999357739
Iteration:      4,   Func. Count:     32,   Neg. LLF: 47950.34106097754
Iteration:      5,   Func. Count:     40,   Neg. LLF: 520034.97158894496
Iteration:      6,   Func. Count:     48,   Neg. LLF: 63029.13501302644
Iteration:      7,   Func. Count:     56,   Neg. LLF: 174524.6051369701
Iteration:      8,   Func. Count:     64,   Neg. LLF: 55476.26006505178
Iteration:      9,   Func. Count:     72,   Neg. LLF: 158862.06637208027
Iteration:     10,   Func. Count:     80,   Neg. LLF: 39844.75074684503
Ite

In [30]:
adf_full_month("AMRS")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
   Augmented Dickey-Fuller Results   
Test Statistic                 -3.014
P-value                         0.034
Lags                                5
-------------------------------------

Trend: Constant
Critical Values: -3.43 (1%), -2.86 (5%), -2.57 (10%)
Null Hypothesis: The process contains a unit root.
Alternative Hypothesis: The process is weakly stationary.


In [31]:
t1_to_t2_test("AMRS", "MGPI", 30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Iteration:      1,   Func. Count:      8,   Neg. LLF: 796086.1460148464
Iteration:      2,   Func. Count:     18,   Neg. LLF: 9204694.118006913
Iteration:      3,   Func. Count:     27,   Neg. LLF: 128169.54463506334
Iteration:      4,   Func. Count:     36,   Neg. LLF: 143541.3484736513
Iteration:      5,   Func. Count:     45,   Neg. LLF: 37988.52774370289
Iteration:      6,   Func. Count:     54,   Neg. LLF: 37483.9338759365
Iteration:      7,   Func. Count:     62,   Neg. LLF: 40701.62070393779
Iteration:      8,   Func. Count:     70,   Neg. LLF: 37423.575436324114
Iteration:      9,   Func. Count:     77,   Neg. LLF: 37423.53946324886
Iteration:     10,   Func. Count:     84,   Neg. LLF: 37423.53281471749
Ite

In [91]:
def fat_vol(ticker):
    ticker = full_month(ticker)
    returns = 10000 * (ticker["Close"].pct_change().dropna())
    return ticker, returns, fattails.mad(returns)

In [93]:
def fat_ROI(ticker):
    t, t_returns, t_vol = fat_vol(ticker)
    t_diff = t_returns + t_vol
    t_times = t_diff[t_diff < 0].reset_index()
    t_t = t.reset_index().reset_index()
    #5 minute window
    for i in range(1, 5):
        trimmed = t_t[:-i]
        entries = trimmed.loc[trimmed["Datetime"].isin(t_times["Datetime"])].reset_index(drop=True)
        sell = t_t.loc[t_t["index"].isin(entries["index"] + i)]
        profit = sum(sell["Close"]) - sum(entries["Close"])
    return "Profit for 5 minute window: " + str(profit)



In [94]:
fat_ROI("SEDG")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


'Profit for 5 minute window: 51.41363525390625'